In [30]:
import numpy as np
import pandas as pd
from cleanData import clean
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras 
from sklearn.preprocessing import OneHotEncoder

In [24]:
data = pd.read_csv('train.csv')
data = clean(data).drop('Credit_Score', axis=1)
data.Score = data.Score.map({3:2, 2:1, 1:0})
data.head()

C:\Users\maule\AppData\Local\Temp\ipykernel_19968\1347739257.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('train.csv')


,Monthly_Inhand_Salary,Num_Credit_Card,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Monthly_Balance,Score
ID,,,,,,,,,,,
0x1602,1824.843333,4,4,3,7.0,4.0,809.98,26.822620,22.0,312.494089,2
0x1603,1824.843333,4,4,-1,7.0,4.0,809.98,31.944960,22.0,284.629162,2
0x1604,1824.843333,4,4,3,7.0,4.0,809.98,28.609352,22.0,331.209863,2
0x1605,1824.843333,4,4,5,4.0,4.0,809.98,31.377862,22.0,223.451310,2
0x1606,1824.843333,4,4,6,4.0,4.0,809.98,24.797347,22.0,341.489231,2


In [9]:
xTrain, xTest, yTrain, yTest = train_test_split(data.drop('Score', axis=1), data.pop('Score'), test_size=0.2)

In [10]:
model = keras.Sequential([
    keras.layers.Dense(1024, activation=keras.layers.LeakyReLU(alpha=0.01), input_shape=(xTrain.shape[1],)),
    keras.layers.Dense(512, activation='tanh'),
    keras.layers.Dense(256, activation='sigmoid'),
    keras.layers.Dense(3, activation='softmax')
])

In [35]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [33]:
pd.get_dummies(yTrain)

,1,2,3
ID,,,
0x16927,True,False,False
0x242db,False,False,True
0x2046,True,False,False
0xd7fe,False,True,False
0x1d117,True,False,False
...,...,...,...
0x11df6,False,True,False
0x2369a,True,False,False
0x22311,False,False,True


In [37]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
history = model.fit(xTrain, pd.get_dummies(yTrain, dtype=float), batch_size=2, validation_split=0.2, callbacks=earlyStopping, epochs=500)

Epoch 1/500
22206/32000 [===================>..........] - ETA: 1:36 - loss: nan - accuracy: 0.2915